<a href="https://colab.research.google.com/github/Ramprasad-Group/Sublimation_enthalpy_model/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



This notebook imports model published in the article titled "Accelerated predictions of the sublimation enthalpy of organic materials with machine learning" and offers predictions for the sublimation enthalpy of new candidates. For more detailed information, please refer to https://onlinelibrary.wiley.com/doi/full/10.1002/mgea.84.

In [8]:
#@title Predict the Sublimation Enthalpy of Organic Molecules
#@markdown *Please first input your candidate's SMILES string, and then press on the left button to run.*

#@markdown *This model will check and load the RDkit 2022.9.05 version, then give a prediction for the input SMILES.*

#Clone the repository if not already present
import os
if not os.path.exists("/content/Sublimation_enthalpy_model"):
    !git clone https://github.com/Ramprasad-Group/Sublimation_enthalpy_model
else:
    print("Repository already cloned.")

#Load rdkit 2022.9.05 version
!pip install rdkit==2022.9.05
!pip install numpy==1.23.5 # Downgrade NumPy

#Load necessary packages
import joblib
import os
import pickle
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.preprocessing import StandardScaler
import warnings

# Suppress InconsistentVersionWarning
warnings.filterwarnings("ignore", category=UserWarning)

#Paths to the required files
scaler_path = "/content/Sublimation_enthalpy_model/845scaler.save"
model_path = "/content/Sublimation_enthalpy_model/845model.pkl"
zero_features = "/content/Sublimation_enthalpy_model/all_zero_features.txt"

#Load the scaler
with open(scaler_path, 'rb') as f:
    scaler = joblib.load(f)

#Load the model
with open(model_path, 'rb') as f:
    model = joblib.load(f)

# @markdown 1. Enter a SMILES string:
smiles = "CCCCCCCC" # @param {type:"string"}

def compute_descriptors(smiles):
    """
    Compute RDKit molecular descriptors for a given SMILES string.
    Parameters:
        smiles (str): The SMILES string of the molecule.
    Returns:
        np.array: A NumPy array of molecular descriptors.
    """
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError(f"Invalid SMILES string: {smiles}")

    # Extract descriptors
    descriptor_values = [func(mol) for _, func in Descriptors.descList]
    return np.array(descriptor_values)

def get_smiles_input():
    """Request user input for a SMILES string."""
    return input("Enter a SMILES string: ")

def predict_sublimation_enthalpy(smiles):
    """Predict the sublimation enthalpy for a given SMILES string."""
    try:
        # Compute molecular descriptors
        descriptors = compute_descriptors(smiles)

        # Reshape and normalize the descriptors using the scaler
        # Load the list of zero features
        with open(zero_features, 'r') as f:
            zero_features_list = f.read().splitlines()

        # Convert the list of zero features to indices
        #zero_indices = [Descriptors.descList.index((name, func)) for name, func in Descriptors.descList if name in zero_features_list]
        zero_indices = [int(feature) for feature in zero_features_list]

        # Drop the zero features from the descriptors
        descriptors_filtered = np.delete(descriptors, zero_indices)

        # Reshape and normalize the descriptors using the scaler
        descriptors_normalized = scaler.transform([descriptors_filtered])

        # Use the model to predict the sublimation enthalpy and its uncertainty
        prediction, uncertainty = model.predict(descriptors_normalized, return_std=True)
        return prediction[0], uncertainty[0]
    except Exception as e:
        print(f"Error: {e}")
        return None, None


if __name__ == "__main__":

    # Predict the sublimation enthalpy
    enthalpy, uncertainty = predict_sublimation_enthalpy(smiles)

    # Display the result
    if enthalpy is not None and uncertainty is not None:
        print(f"Predicted Sublimation Enthalpy: {enthalpy:.2f} kJ/mol")
        print(f"Prediction Uncertainty: ±{uncertainty:.2f} kJ/mol")
    else:
        print("Failed to predict sublimation enthalpy.")

Repository already cloned.
Predicted Sublimation Enthalpy: 67.24 kJ/mol
Prediction Uncertainty: ±12.67 kJ/mol


In [12]:
#@title Batch Prediction for the Sublimation Enthalpy prediction of Organic Molecules

#@markdown *In this section, you can utilize our model to predict the Sublimation Enthalpy of a number of Organic Molecules. Please first upload your candidate's SMILES string in CSV format ("candidate.csv"). Once you have prepared your CSV file, upload it to Colab using the icon in the left column and then click the left button to initiate the process. To illustrate this, We will use a sample candidate dataset. Users can subsequently download the predicted results, which are saved in the 'predicted.csv' file accessible through the icon in the left column.*

#@markdown *This model will check and load the RDkit 2022.9.05 version, then give a prediction for the input SMILES.*

#Clone the repository if not already present
import os
if not os.path.exists("/content/Sublimation_enthalpy_model"):
    !git clone https://github.com/Ramprasad-Group/Sublimation_enthalpy_model
else:
    print("Repository already cloned.")


#Load rdkit 2022.9.05 version
!pip install rdkit==2022.9.05
!pip install numpy==1.23.5 # Downgrade NumPy

#Load necessary packages
import joblib
import os
import pickle
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.preprocessing import StandardScaler
import warnings

# Suppress InconsistentVersionWarning
warnings.filterwarnings("ignore", category=UserWarning)

#Paths to the required files
scaler_path = "/content/Sublimation_enthalpy_model/845scaler.save"
model_path = "/content/Sublimation_enthalpy_model/845model.pkl"
zero_features = "/content/Sublimation_enthalpy_model/all_zero_features.txt"

#Load the scaler
with open(scaler_path, 'rb') as f:
    scaler = joblib.load(f)

#Load the model
with open(model_path, 'rb') as f:
    model = joblib.load(f)


def compute_descriptors(smiles):

    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError(f"Invalid SMILES string: {smiles}")

    # Extract descriptors
    descriptor_values = [func(mol) for _, func in Descriptors.descList]
    return np.array(descriptor_values)


def predict_sublimation_enthalpy(smiles):
    """Predict the sublimation enthalpy for a given SMILES string."""
    try:
        # Compute molecular descriptors
        descriptors = compute_descriptors(smiles)

        # Reshape and normalize the descriptors using the scaler
        # Load the list of zero features
        with open(zero_features, 'r') as f:
            zero_features_list = f.read().splitlines()

        # Convert the list of zero features to indices
        #zero_indices = [Descriptors.descList.index((name, func)) for name, func in Descriptors.descList if name in zero_features_list]
        zero_indices = [int(feature) for feature in zero_features_list]

        # Drop the zero features from the descriptors
        descriptors_filtered = np.delete(descriptors, zero_indices)

        # Reshape and normalize the descriptors using the scaler
        descriptors_normalized = scaler.transform([descriptors_filtered])

        # Use the model to predict the sublimation enthalpy and its uncertainty
        prediction, uncertainty = model.predict(descriptors_normalized, return_std=True)
        return prediction[0], uncertainty[0]
    except Exception as e:
        print(f"Error: {e}")
        return None, None


if __name__ == "__main__":

    if os.path.exists("/content/candidate.csv"):
        pass
    else:
        !cp /content/Sublimation_enthalpy_model/Test.csv /content/candidate.csv

    ### Read the predict csv file###
    pred = pd.read_csv("/content/candidate.csv")
    smiles = pred['smiles'].to_numpy()

    # Process each SMILES
    predictions = []
    uncertainty = []
    for smiles in pred['smiles']:
        enthalpy, uncertainty_value = predict_sublimation_enthalpy(smiles)
        predictions.append(enthalpy)
        uncertainty.append(uncertainty_value)

    # Create a pandas DataFrame for predicted sublimation enthalpy
    pred_df = pd.DataFrame()
    pred_df['smiles'] = pred['smiles']
    pred_df['predicted'] = predictions
    pred_df['uncertainty'] = uncertainty
    pred_df.to_csv('/content/predicted.csv')
    print(pred_df)

Repository already cloned.
                smiles  predicted  uncertainty
0              C1CCCC1  44.984163    12.631777
1                 CCCC  38.839736    12.111194
2               CCCCCC  50.619415    12.169698
3                  C=C  36.606528    14.624640
4  C1CCC(=C2CCCCC2)CC1  88.378671    12.671322
